In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#mport pandas and numpy
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [3]:
#reading the train and test dataset
train = pd.read_csv('/content/drive/My Drive/Train (dsnbootcamp).csv')
test = pd.read_csv('/content/drive/MyDrive/Test (dsnbootcamp).csv')

train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [4]:
#dropping the applicant_id and default_status column
features = train.columns.drop(["Applicant_ID", "default_status"])
#flling in the missing column 
X = pd.get_dummies(train[features])
X.fillna(-999, inplace=True)
y = train.default_status
test_x = pd.get_dummies(test[features].copy())
test_x.fillna(-999, inplace=True)

In [5]:
test_x.shape

(24000, 51)

In [6]:
#installingcatboost
pip install catboost

     |████████████████████████████████| 65.7MB 53kB/s 


In [7]:
#importing the metrics to be used in our voting classifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [8]:
rf = RandomForestClassifier(n_estimators=700, n_jobs=-1)
lr = LogisticRegression()
xg = XGBClassifier(n_jobs=-1)
cat = CatBoostClassifier(verbose=False, thread_count=-1, loss_function="MultiClass")
lg = LGBMClassifier(n_jobs=-1)
minmax = MinMaxScaler()

In [9]:
#
def model_auc(model):
    train_auc = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    test_auc = roc_auc_score(y_test, model.predict_proba(x_test)[:,1])
    print('Train AUC: {} \t Val AUC: {}'.format(train_auc,test_auc))
    return test_auc

In [10]:
seed = 32

In [11]:

skf = StratifiedKFold(n_splits=25, shuffle=True, random_state=seed)
scores = []
preds = []
#
vc = VotingClassifier(estimators=[('cat', CatBoostClassifier(verbose=False, random_state=seed, thread_count=-1)),
                                  ('cat2', CatBoostClassifier(bagging_temperature=0.8609, border_count=116, depth=5, 
                                                              iterations=270, l2_leaf_reg=3, learning_rate=0.070856, 
                                                              random_strength=0.0882067, scale_pos_weight=0.390675, 
                                                              verbose=False, random_state=seed, thread_count=-1)),
                                  ('lg', LGBMClassifier(n_jobs=-1, random_state=seed)),
                                  ('rf', RandomForestClassifier(n_estimators=500, n_jobs=-1))], voting='soft')


i = 1
for train, test in skf.split(X, y):
    pipe = Pipeline([('scaler', minmax), ('model', vc)])
    x_train, x_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
    pipe.fit(x_train, y_train)
    print('Number of splits trained {}'.format(i))
    score = model_auc(pipe)
    pred = pipe.predict_proba(test_x)[:, 1]
    scores.append(score)
    preds.append(pred)
    i += 1
    print('\n ----------------------' * 2)

np.mean(scores)

Number of splits trained 1
Train AUC: 0.9688742721350108 	 Val AUC: 0.8449120808959292

 ----------------------
 ----------------------
Number of splits trained 2
Train AUC: 0.9688512613297415 	 Val AUC: 0.8401925764870322

 ----------------------
 ----------------------
Number of splits trained 3
Train AUC: 0.9690514547742979 	 Val AUC: 0.836471760625356

 ----------------------
 ----------------------
Number of splits trained 4
Train AUC: 0.969026674051004 	 Val AUC: 0.8258453262238789

 ----------------------
 ----------------------
Number of splits trained 5
Train AUC: 0.9688371281788347 	 Val AUC: 0.8610712067091162

 ----------------------
 ----------------------
Number of splits trained 6
Train AUC: 0.9690063854769462 	 Val AUC: 0.85056125002157

 ----------------------
 ----------------------
Number of splits trained 7
Train AUC: 0.9687896958725348 	 Val AUC: 0.8441183068454384

 ----------------------
 ----------------------
Number of splits trained 8
Train AUC: 0.968825707904

0.8415814939042258

In [12]:
prediction = np.mean(preds, axis=0)

In [14]:
#reading the sample submission file
sample_sub = pd.read_csv("/content/drive/My Drive/SampleSubmission (dsnbootcamp).csv")
sample_sub.head()

,Applicant_ID,default_status
0,Apcnt_1000032,1
1,Apcnt_1000048,1
2,Apcnt_1000052,1
3,Apcnt_1000076,1
4,Apcnt_1000080,1


In [15]:
#assigning prediction to the default status of the sample sub
sample_sub.default_status = prediction

In [16]:
sample_sub.to_csv("sub7", index=False)